In [62]:
from networkx import read_gpickle
graph_path = '/tmp/estimate-electricity-cost-by-technology-from-population/results/20/infrastructure_graph.pkl'

In [63]:
g = read_gpickle(graph_path)

In [64]:
from geopy.distance import vincenty as get_distance
node1_id, node2_id = g.edges_iter().next()
node1 = g.node[node1_id]
node1_xy = node1['latitude'], node1['longitude']
node2 = g.node[node2_id]
node2_xy = node2['latitude'], node2['longitude']
x = get_distance(node1_xy, node2_xy)

In [65]:
x.meters

346610.4534300535

In [56]:
[(1, 1) for x in xrange(3)]

[(1, 1), (1, 1), (1, 1)]

In [57]:
import math
math.sqrt(2)

1.4142135623730951

In [59]:
"""
# Get sum of mv line distance
grid_mv_line_adjusted_length_in_meters = 0
grid_mv_line_length_adjustment_factor = 1.4
for node1_id, node2_id in g.edges_iter():
    nodes = [g.node[x] for x in (node1_id, node2_id)]
    node_lls = [(x['latitude'], x['longitude']) for x in nodes]
    straight_length_in_meters = get_distance(*node_lls).meters
    adjusted_length_in_meters = grid_mv_line_length_adjustment_factor * \
        straight_length_in_meters
    g.edge[node1_id][node2_id]['grid_mv_line_straight_length_in_meters'] = \
        straight_length_in_meters
    g.edge[node1_id][node2_id]['grid_mv_line_adjusted_length_in_meters'] = \
        adjusted_length_in_meters
    grid_mv_line_adjusted_length_in_meters += adjusted_length_in_meters
grid_mv_line_adjusted_length_in_meters
""";

In [70]:
[x for x in g.node[1] if x.startswith('grid_mv_line')]

['grid_mv_line_discounted_cost_per_meter',
 'grid_mv_line_installation_lm_cost_per_meter',
 'grid_mv_line_maintenance_lm_cost_per_meter_per_year',
 'grid_mv_line_replacement_lm_cost_per_meter_per_year']

In [75]:
def estimate_grid_mv_line_length_in_meters(infrastructure_graph, line_length_adjustment_factor):
    grid_mv_line_adjusted_length_in_meters = 0
    g = infrastructure_graph
    for node1_id, node2_id in g.edges_iter():
        nodes = [g.node[x] for x in (node1_id, node2_id)]
        node_lls = [(x['latitude'], x['longitude']) for x in nodes]
        straight_length_in_meters = get_distance(*node_lls).meters
        adjustment_factor = np.mean([g.node[x].get(
            'line_length_adjustment_factor', line_length_adjustment_factor,
        ) for x in [node1_id, node2_id]])
        adjusted_length_in_meters = adjustment_factor * straight_length_in_meters
        halved_adjusted_length_in_meters = adjusted_length_in_meters / 2.
        grid_mv_line_adjusted_length_in_meters += adjusted_length_in_meters
        g.edge[node1_id][node2_id]['grid_mv_line_adjusted_length_in_meters'] = adjusted_length_in_meters
        g.node[node1_id]['grid_mv_line_adjusted_length_in_meters'] = halved_adjusted_length_in_meters
        g.node[node2_id]['grid_mv_line_adjusted_length_in_meters'] = halved_adjusted_length_in_meters
    return [
        ('grid_mv_line_adjusted_length_in_meters', grid_mv_line_adjusted_length_in_meters),
    ]

estimate_grid_mv_line_length_in_meters(g, 1.4)

[('grid_mv_line_adjusted_length_in_meters', 1189496.749735076)]

In [47]:
g.edge[1]

{0: {'distance_in_meters': 346610.4534300535,
  'grid_mv_line_adjusted_length_in_meters': 485254.63480207487,
  'grid_mv_line_straight_length_in_meters': 346610.4534300535},
 2: {'distance_in_meters': 503030.08209500083,
  'grid_mv_line_adjusted_length_in_meters': 704242.1149330011,
  'grid_mv_line_straight_length_in_meters': 503030.08209500083}}

In [48]:
g.edges_iter(data=True).next()

(0,
 1,
 {'distance_in_meters': 346610.4534300535,
  'grid_mv_line_adjusted_length_in_meters': 485254.63480207487,
  'grid_mv_line_straight_length_in_meters': 346610.4534300535})

In [41]:
[x for x in g.node[node1_id].keys() if x.startswith('grid_mv_line')]

['grid_mv_line_discounted_cost_per_meter',
 'grid_mv_line_installation_lm_cost_per_meter',
 'grid_mv_line_maintenance_lm_cost_per_meter_per_year',
 'grid_mv_line_replacement_lm_cost_per_meter_per_year']

In [42]:
import numpy as np

In [43]:
np.mean([2, 3])

2.5

In [44]:
g.node[1]['grid_mv_line_discounted_cost_per_meter']

25.325291491610727

In [49]:
grid_mv_line_cost = 0
for node1_id, node2_id, edge_d in g.edges_iter(data=True):
    grid_mv_line_average_discounted_cost_per_meter = np.mean([
        g.node[x]['grid_mv_line_discounted_cost_per_meter'] for x in [node1_id, node2_id]])
    distance_in_meters = edge_d['grid_mv_line_adjusted_length_in_meters']
    grid_mv_line_cost += grid_mv_line_average_discounted_cost_per_meter * distance_in_meters
grid_mv_line_cost

30124351.915364332

In [ ]:
# Get discounted cost of mv line
def estimate_grid_mv_line_cost(
        grid_mv_line_adjusted_length_in_meters,
        grid_mv_line_installation_lm_cost_per_meter,
        grid_mv_line_maintenance_lm_cost_per_meter_per_year,
        grid_mv_line_replacement_lm_cost_per_meter_per_year):
    installation_lm_cost = grid_mv_line_installation_lm_cost_per_meter * \
        grid_mv_line_adjusted_length_in_meters
    maintenance_lm_cost_per_year = grid_mv_line_maintenance_lm_cost_per_meter_per_year * \
        grid_mv_line_adjusted_length_in_meters
    replacement_lm_cost_per_year = grid_mv_line_replacement_lm_cost_per_meter_per_year * \
        grid_mv_line_adjusted_length_in_meters
    return [
        ('installation_lm_cost', installation_lm_cost),
        ('maintenance_lm_cost_per_year', maintenance_lm_cost_per_year),
        ('replacement_lm_cost_per_year', replacement_lm_cost_per_year),
    ]

d = prepare_component_cost_by_year([
    ('mv_line', estimate_grid_mv_line_cost),
], kw, prefix='grid_')
grid_mv_line_discounted_cost = compute_discounted_cash_flow(
    d.pop('cost_by_year'),
    kw['financing_year'],
    kw['discount_rate_as_percent_of_cash_flow_per_year'])

In [ ]:
# Add to discounted grid cost
# Compute levelized grid cost

In [ ]:
# For each node, select least expensive technology
# Sum costs for selected technology across all nodes

In [ ]:
# Sum all discounted costs
# Compute giant levelized costs for selected technology

In [ ]:
# Get counts
# Render selected technology as color in map